In [1]:
%%HTML
<style>
    body {
        --vscode-font-family: "KoddiUD 온고딕"
    }
</style>

# Ch 1. 장바구니 분석

- 장바구니 분석 = 연관분석 = (Market) Basket Analysis = Association Analysis

**장바구니 분석**

- 고객의 결제 데이터 안에서 특정한 패턴을 찾아내는 분석 기법
- 어떤 상품들이 함께 구매되는지 **연관성**에 주목하는 방법
- 특정 상품을 집어든 고객이 함께 구매할 가능성이 높은 상품이 무엇인지 파악한 후 구매 동선을 설계하거나 (연관 규칙 마이닝)
- 오늘 특정 상품을 구매한 고객이 다음 날 어떤 상품을 구매할 지 예측해 고객 관계 관리에 활용 (순차 패턴 마이닝)

**장바구니 분석의 목적**

1. 효율적인 마케팅 전략을 통한 매출 극대화
2. 장기적인 고객 충성도 강화

# Ch 2. 결제 데이터

- 장바구니 분석은 대부분 결제 데이터를 기반으로 함
- 결제 데이터는 고객이 상품이나 서비스를 구매한 기록

1) 누가 (Who) : 이 상품을 구매한 사람은 누구인가

2) 언제 (When) : 언제 구매가 일어났는가

3) 어디서 (Where) : 어디에서 구매가 일어났는가

4) 어떻게 (How) : 어떻게 결제했는가

5) 무엇을 (What) : 어떤 상품을 구매했는가 - 상품별 연관성

*장바구니 분석 : 이 중에서 무엇을 샀는지에 초점을 맞춰 상품별 조합을 탐구하는 분석 방법론*

1) 결제 ID : 결제 건마다 고유한 ID를 붙여 저장. 영수증 번호.

2) 상품 정보 : 고객이 구매한 상품

3) 고객 정보

4) 거래 시점

In [2]:
# 라이브러리 불러오기
import pandas as pd

In [3]:
# 데이터 불러오기
df = pd.read_csv('retail_data.csv')
df.head()

,OrderID,StockCode,ProdName,Quantity,OrderDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0


**변수 설명**

- OrderID : 결제 ID
- StockCode : 상품 코드
- ProdName : 상품 이름
- Quantity : 상품 수량
- OrderDate : 거래 날짜
- Unit Price : 단위 가격
- CustomerID : 고객 ID

In [4]:
df2 = df[['OrderID', 'ProdName']]
df2.head()

,OrderID,ProdName
0,536365,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,WHITE METAL LANTERN
2,536365,CREAM CUPID HEARTS COAT HANGER
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,RED WOOLLY HOTTIE WHITE HEART.


In [5]:
basket_df = df2.groupby('OrderID')['ProdName'].apply(list).reset_index()
basket_df.head()

,OrderID,ProdName
0,536365,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET..."
1,536366,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA..."
2,536367,"[ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO..."
3,536368,"[JAM MAKING SET WITH JARS, RED COAT RACK PARIS..."
4,536369,[BATH BUILDING BLOCK WORD]


# Ch 3. 연관 규칙 마이닝

- 연관 규칙 마이닝 : 데이터 안에서 아이템 간의 상호 관련성을 탐색해 유의미한 정보를 추출해 내는 장바구니 분석 방법론
- 연관 규칙 : 상품 간의 연관성을 나타내는 규칙
- 예를 들어, 만약 고객이 우유와 달걀을 함께 구매하면, 빵도 함께 구매할 가능성이 높다는 연관 규칙이 있을 때,
- 우유와 달걀을 구매했기 때문에 (원인), 빵도 구매하게 되는 것 (결과) 이 아니라
- 우유와 달걀 그리고 빵이 구매 측면의 연관성을 보인다.

## 좋은 연관 규칙을 찾는 법

(1) 지지도 (Support)

- 규칙이 얼마나 일반적인지를 평가하는 지표.
- 주어진 데이터 안에서 규칙이 얼마나 자주 등장하는지 그 빈도를 계산
- X -> Y의 관계를 갖는 규칙에서 지지도는 X와 Y가 함께 포함된 거래 수를 전체 거래 수로 나눈 값
- 지지도 값은 1에 가까울수록 해당 규칙이 데이터 안에서 빈번하게 발생
- 만약 지지도 값이 너무 낮은 규칙이라면 몇몇 고객에게서만 관찰된 특이 행동일 가능성이 높기 때문에 범용성이 떨어짐

(2) 신뢰도 (Confidence)

- 규칙이 얼마나 믿을만한지 보여주는 지표
- X -> Y도 구매한다는 규칙이 있을 때 X를 구매하는 사람 중 이 규칙대로 구매하는 사람이 얼마나 되는지
- 신뢰도는 조건 (IF) 부분에 해당하는 경우의 수를 분모로 두고 있기 때문에 X -> Y 관계인지 Y -> X 관계이냐에 따라 수치가 달라지고 추천의 *방향성*까지 고려 가능

(3) 향상도 (Lift)

- X를 구매하면 Y도 구매한다는 규칙이 있을 때 X를 구매하는 행위가 Y를 구매하는 확률에 어떤 영향을 주는지 나타내는 지표

## Apriori 알고리즘

- Brute Force (무차별 탐색) : 무차별적으로 모든 경우의 수를 다 조사하는 방법
- Apriori : 라틴어로 '선험적인, 연역적인'. 상위 조합에서부터 차례로 스캔하면서 특정 조합이 자주 발생하지 않는다면 이의 결과물로 탄생한 후속 조합들까지 모두 후보에서 배제하는 방식
- Apriori 알고리즘을 활용하면 하나의 조합만 검사하고도 이로부터 파생된 다른 조합들까지 후보에서 배제할 수 있게 됨
- 빈발 항목 집합 (Frequent Itemset) :특정 지지도 이상의 조합

### 파이썬으로 Apriori 알고리즘 구현하기

- mlxtend (Machine Learning Extenstions)

In [6]:
!pip install mlxtend

In [7]:
# 라이브러리 불러오기
import pandas as pd

In [8]:
# 데이터 불러오기
df = pd.read_csv('retail_data.csv')
df.head()

,OrderID,StockCode,ProdName,Quantity,OrderDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0


In [9]:
# 주문 번호별로 구매한 상품을 묶기
basket_df = df.groupby('OrderID')['ProdName'].apply(list).reset_index()
basket_df.head()

,OrderID,ProdName
0,536365,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET..."
1,536366,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA..."
2,536367,"[ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO..."
3,536368,"[JAM MAKING SET WITH JARS, RED COAT RACK PARIS..."
4,536369,[BATH BUILDING BLOCK WORD]


In [10]:
basket_df.shape

(270, 2)

In [11]:
# 라이브러리 불러오기 (TransactionEncoder : 데이터를 이진 형태로 변환)
from mlxtend.preprocessing import TransactionEncoder

In [12]:
basket_df.head()

,OrderID,ProdName
0,536365,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET..."
1,536366,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA..."
2,536367,"[ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO..."
3,536368,"[JAM MAKING SET WITH JARS, RED COAT RACK PARIS..."
4,536369,[BATH BUILDING BLOCK WORD]


In [13]:
te = TransactionEncoder()
te_result = te.fit_transform(basket_df['ProdName'])
te_result

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [14]:
te_df = pd.DataFrame(te_result, columns=te.columns_)
te_df.head()

,4 PURPLE FLOCK DINNER CANDLES,SET 2 TEA TOWELS I LOVE LONDON,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,...,"WRAP, BILLBOARD FONTS DESIGN",YELLOW BREAKFAST CUP AND SAUCER,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,ZINC FINISH 15CM PLANTER POTS,ZINC METAL HEART DECORATION,ZINC WILLIE WINKIE CANDLE STICK
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [15]:
te_df.shape

(270, 1343)

(Apriori 알고리즘 구현)

- mlxtend 라이브러리의 apriori 함수를 이용하여 apriori 알고리즘 구현
- 최소 지지도 0.05 (일반적으로 1%~5%)

In [16]:
# 라이브러리 불러오기
from mlxtend.frequent_patterns import apriori

In [17]:
apriori(te_df, min_support=0.05)

,support,itemsets
0,0.051852,(60)
1,0.062963,(84)
2,0.066667,(339)
3,0.066667,(511)
4,0.066667,(544)
...,...,...
3358,0.051852,"(642, 1315, 1252, 1253, 1286, 1311, 1289, 976,..."
3359,0.051852,"(1315, 1252, 1253, 1286, 1311, 1289, 974, 976,..."
3360,0.051852,"(642, 1315, 1252, 1253, 1286, 1289, 974, 976, ..."
3361,0.051852,"(642, 1315, 1252, 1253, 1286, 1311, 1289, 974,..."


In [18]:
apriori(te_df, min_support=0.05, use_colnames=True)  # 상품명으로 불러옴

,support,itemsets
0,0.051852,(ALARM CLOCK BAKELIKE GREEN)
1,0.062963,(ASSORTED COLOUR BIRD ORNAMENT)
2,0.066667,(CREAM CUPID HEARTS COAT HANGER)
3,0.066667,(GLASS STAR FROSTED T-LIGHT HOLDER)
4,0.066667,(HAND WARMER BIRD DESIGN)
...,...,...
3358,0.051852,"(WOODEN FRAME ANTIQUE WHITE , VINTAGE BILLBOAR..."
3359,0.051852,"(WOODEN FRAME ANTIQUE WHITE , VINTAGE BILLBOAR..."
3360,0.051852,"(WOODEN FRAME ANTIQUE WHITE , VINTAGE BILLBOAR..."
3361,0.051852,"(WOODEN FRAME ANTIQUE WHITE , VINTAGE BILLBOAR..."


In [19]:
frequent_itemsets = apriori(te_df, min_support=0.05, use_colnames=True)  # 빈발 집합으로 저장
frequent_itemsets

,support,itemsets
0,0.051852,(ALARM CLOCK BAKELIKE GREEN)
1,0.062963,(ASSORTED COLOUR BIRD ORNAMENT)
2,0.066667,(CREAM CUPID HEARTS COAT HANGER)
3,0.066667,(GLASS STAR FROSTED T-LIGHT HOLDER)
4,0.066667,(HAND WARMER BIRD DESIGN)
...,...,...
3358,0.051852,"(WOODEN FRAME ANTIQUE WHITE , VINTAGE BILLBOAR..."
3359,0.051852,"(WOODEN FRAME ANTIQUE WHITE , VINTAGE BILLBOAR..."
3360,0.051852,"(WOODEN FRAME ANTIQUE WHITE , VINTAGE BILLBOAR..."
3361,0.051852,"(WOODEN FRAME ANTIQUE WHITE , VINTAGE BILLBOAR..."


In [20]:
num_itemsets = len(frequent_itemsets)
print("항목 세트의 개수:", num_itemsets)

항목 세트의 개수: 3363


(연관 규칙 추출 및 평가)

- mlxtend 라이브러리의 association_rules() 함수를 적용하여 규칙 평가하기 위한 지표 계산
- 최소 신뢰도 0.8 (일반적으로 50%~80%)

In [21]:
# 라이브러리 불러오기
from mlxtend.frequent_patterns import association_rules

- antecedents가 IF부분이고, consequents가 THEN에 대항하는 부분
- 예) 규칙 : "우유 -> 빵"
  - Antecedent Support : "우유"가 포함된 거래가 얼마나 자주 발생하는지
  - Consequent Support : "빵"이 포함된 거래가 얼마나 자주 발생하는지
  - Support : "우유"와 "빵"이 함께 나타난 거래가 얼마나 자주 발생하는지


In [22]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8, num_itemsets=61)
rules.sort_values(by='lift', ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
185773,"(CREAM CUPID HEARTS COAT HANGER, VINTAGE BILLB...","(GLASS STAR FROSTED T-LIGHT HOLDER, WOODEN PIC...",0.051852,0.051852,0.051852,1.0,19.285714,1.0,0.049163,inf,1.0,1.0,1.0,1.0
84898,"(CREAM CUPID HEARTS COAT HANGER, RED WOOLLY HO...","(KNITTED UNION FLAG HOT WATER BOTTLE, WOODEN P...",0.051852,0.051852,0.051852,1.0,19.285714,1.0,0.049163,inf,1.0,1.0,1.0,1.0
84897,"(KNITTED UNION FLAG HOT WATER BOTTLE, CREAM CU...","(WOODEN PICTURE FRAME WHITE FINISH, RED WOOLLY...",0.051852,0.051852,0.051852,1.0,19.285714,1.0,0.049163,inf,1.0,1.0,1.0,1.0
84896,"(CREAM CUPID HEARTS COAT HANGER, VINTAGE BILLB...","(KNITTED UNION FLAG HOT WATER BOTTLE, WOODEN P...",0.051852,0.051852,0.051852,1.0,19.285714,1.0,0.049163,inf,1.0,1.0,1.0,1.0
178612,"(WOODEN FRAME ANTIQUE WHITE , CREAM CUPID HEAR...","(WOODEN PICTURE FRAME WHITE FINISH, VINTAGE BI...",0.051852,0.051852,0.051852,1.0,19.285714,1.0,0.049163,inf,1.0,1.0,1.0,1.0


In [23]:
help(association_rules)

Help on function association_rules in module mlxtend.frequent_patterns.association_rules:

association_rules(df: pandas.core.frame.DataFrame, num_itemsets: int, df_orig: Optional[pandas.core.frame.DataFrame] = None, null_values=False, metric='confidence', min_threshold=0.8, support_only=False, return_metrics: list = ['antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'representativity', 'leverage', 'conviction', 'zhangs_metric', 'jaccard', 'certainty', 'kulczynski']) -> pandas.core.frame.DataFrame
    Generates a DataFrame of association rules including the
    metrics 'score', 'confidence', and 'lift'

    Parameters
    -----------
    df : pandas DataFrame
      pandas DataFrame of frequent itemsets
      with columns ['support', 'itemsets']

    df_orig : pandas DataFrame (default: None)
      DataFrame with original input data. Only provided when null_values exist

    num_itemsets : int
      Number of transactions in original input data

    null_values

## FP-Growth 알고리즘

- Apriori 알고리즘은 무차별적으로 모든 경우의 수를 다 조사하는 Brute Force (무차별 탐색) 방법의 단점을 극복하기 위해 고안된 방안이지만, Apriori 알고리즘도 데이터가 방대해질수록 속도가 점점 느려진다는 단점 (각 단계마다 데이터셋을 매번 다시 스캔해서 후보 집합을 만들어야 하기 때문)

- FP-Growth (Frequent Pattern Growth) 알고리즘 : 최소 지지도 이상인 아이템에 주목하는 Apriori 알고리즘의 접근 방법을 가져오되, *트리 구조*를 활용해 속도를 개선시킨 방법론

### 파이썬으로 FP-Growth 알고리즘 구현하기

In [24]:
# 라이브러리 불러오기
import pandas as pd

In [25]:
# 데이터 불러오기
df = pd.read_csv('retail_data.csv')
df.head()

,OrderID,StockCode,ProdName,Quantity,OrderDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0


In [26]:
# OrderID별로 ProdName을 리스트화
basket_df = df.groupby('OrderID')['ProdName'].apply(list).reset_index()
basket_df.head()

,OrderID,ProdName
0,536365,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET..."
1,536366,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA..."
2,536367,"[ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO..."
3,536368,"[JAM MAKING SET WITH JARS, RED COAT RACK PARIS..."
4,536369,[BATH BUILDING BLOCK WORD]


In [27]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

In [28]:
# TransactionEncoder를 활용해 데이터 이진 변환
te = TransactionEncoder()
te_result = te.fit_transform(basket_df['ProdName'])
te_result

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [29]:
te_df = pd.DataFrame(te_result, columns=te.columns_)
te_df.head()

,4 PURPLE FLOCK DINNER CANDLES,SET 2 TEA TOWELS I LOVE LONDON,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,...,"WRAP, BILLBOARD FONTS DESIGN",YELLOW BREAKFAST CUP AND SAUCER,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,ZINC FINISH 15CM PLANTER POTS,ZINC METAL HEART DECORATION,ZINC WILLIE WINKIE CANDLE STICK
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
# 빈발 항목 집합 (최소 지지도 : 0.06)
frequent_itemsets = fpgrowth(te_df, min_support=0.06, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.137037,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.103704,(RED WOOLLY HOTTIE WHITE HEART.)
2,0.096296,(SET 7 BABUSHKA NESTING BOXES)
3,0.085185,(KNITTED UNION FLAG HOT WATER BOTTLE)
4,0.066667,(CREAM CUPID HEARTS COAT HANGER)
...,...,...
56,0.062963,"(KNITTED UNION FLAG HOT WATER BOTTLE, RED WOOL..."
57,0.062963,"(GLASS STAR FROSTED T-LIGHT HOLDER, WHITE HANG..."
58,0.062963,"(HAND WARMER RED POLKA DOT, HAND WARMER UNION ..."
59,0.062963,"(WHITE HANGING HEART T-LIGHT HOLDER, WOOD 2 DR..."


In [31]:
# Corrected code with the DataFrame passed directly
ar = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8, num_itemsets=61)

In [32]:
ar = ar.sort_values(by=['lift', 'confidence'], ascending=False)
ar.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
122,"(KNITTED UNION FLAG HOT WATER BOTTLE, WHITE ME...","(RED WOOLLY HOTTIE WHITE HEART., GLASS STAR FR...",0.062963,0.062963,0.062963,1.0,15.882353,1.0,0.058999,inf,1.0,1.0,1.0,1.0
123,"(KNITTED UNION FLAG HOT WATER BOTTLE, GLASS ST...","(RED WOOLLY HOTTIE WHITE HEART., WHITE METAL L...",0.062963,0.062963,0.062963,1.0,15.882353,1.0,0.058999,inf,1.0,1.0,1.0,1.0
124,"(RED WOOLLY HOTTIE WHITE HEART., WHITE METAL L...","(KNITTED UNION FLAG HOT WATER BOTTLE, GLASS ST...",0.062963,0.062963,0.062963,1.0,15.882353,1.0,0.058999,inf,1.0,1.0,1.0,1.0
125,"(RED WOOLLY HOTTIE WHITE HEART., GLASS STAR FR...","(KNITTED UNION FLAG HOT WATER BOTTLE, WHITE ME...",0.062963,0.062963,0.062963,1.0,15.882353,1.0,0.058999,inf,1.0,1.0,1.0,1.0
135,"(WHITE HANGING HEART T-LIGHT HOLDER, KNITTED U...","(RED WOOLLY HOTTIE WHITE HEART., WHITE METAL L...",0.062963,0.062963,0.062963,1.0,15.882353,1.0,0.058999,inf,1.0,1.0,1.0,1.0


(Apriori 알고리즘 vs. FP-Growth 알고리즘)

1. Apriori 알고리즘

In [34]:
import time
start = time.time()  # 시작 시간 저장
apriori(te_df, min_support=0.01, use_colnames=True)
print("소요 시간:", time.time() - start)

소요 시간: 3.0208640098571777


2. FP-Growth 알고리즘

In [33]:
import time
start = time.time()  # 시작 시간 저장
fpgrowth(te_df, min_support=0.01, use_colnames=True)
print("소요 시간:", time.time() - start)

소요 시간: 132.29857301712036


# 순차 패턴 알고리즘
- 연관 규칙 마이닝(Association Rule Mining)
  - 아이템의 **상호 관련성**을 바탕으로 규칙을 찾는 장바구니 분석 방법론
- 순차 패턴 마이닝(Sequential Pattern Mining)
  - 데이터 안에서 아이템 간의 순차 관계를 탐색해 유의미한 패턴을 찾아내는 분석 방법론
- 예시
  - 연관 규칙 마이닝
    - 어떤 물건들이 함께 구매되는가
    - IF(조건)
      - 만약 고객이 우유와 달걀을 함께 구매하면
    - THEN(결과)
      - 빵도 함께 구매할 가능성이 높음
  - 순차 패턴 마이닝
    - 이 물건 다음에 어떤 물건을 사는가에 대한 분석
    - IF
      - 만약 고객이 금요일에 빵을 구매하면
    - THEN
      - 주말에 우유를 구매할 가능성이 높음
    - 순차 패턴에서는 IF와 THEN 사이에 시차가 존재
  

**순차 패턴 마이닝의 활용 예시**
- 웹/앱 화면 기획
  - 사용자가 화면을 주로 어떤 순서로 방문하는지 파악해 사용자 경험 개선
- 매장 내 동선 최적화
  - 고객이 매장 내에서 어떤 경로로 이동하는지 파악해 매장 구조 최적화
- CRM 마케팅
  - 고객이 이후에 소비할 제품/서비스 예측해 개인화된 마케팅 메세지 발송
- 이상 거래 탐지
  - 금융 거래의 발생 순서를 파악해 이상 거래를 감지
- 자연재해 예측
  - 전조 증상의 발생 순서를 바탕으로 자연재해를 예측

## 시퀀스 데이터(Sequence Data)

- 순차 패턴 마이닝은 시간의 흐름에 따른 패턴을 발견하는 분석 방법
  - &rarr; 순서를 가지는 데이터인 시퀀스 데이터 활용

1. 연관 규칙 마이닝
   - 결제 ID, 구매 상품 정보
2. 순차 패턴 마이닝
   - 결제 ID, 구매 상품 정보, 거래 시점 
   - <>
     - 상품 간 순서가 존재
   - ()
     - 상품 간 순서가 없음

## PrefixSpan 알고리즘(PREFIX-projected Sequential PAttern miNing)

- 특정 아이템을 prefix(접두사)로 두고 해당 아이템으로부터 시작되는 패턴 탐색 방식

In [35]:
from prefixspan import PrefixSpan

In [36]:
# 데이터 준비
sequences = [
    ['a', 'b', 'c', 'd'],
    ['a', 'c', 'd', 'e'],
    ['a', 'b', 'c', 'e'],
    ['b', 'c', 'd', 'e']
]

In [37]:
# PrefixSpan 설정
ps = PrefixSpan(sequences)

In [38]:
# 최소 지지도(minsup) 를 인자로 전달하여 빈발 순차 패턴 추출
result = ps.frequent(minsup=2)

In [39]:
for pattern in result:
  print(pattern)

(3, ['a'])
(2, ['a', 'b'])
(2, ['a', 'b', 'c'])
(3, ['a', 'c'])
(2, ['a', 'c', 'd'])
(2, ['a', 'c', 'e'])
(2, ['a', 'd'])
(2, ['a', 'e'])
(3, ['b'])
(3, ['b', 'c'])
(2, ['b', 'c', 'd'])
(2, ['b', 'c', 'e'])
(2, ['b', 'd'])
(2, ['b', 'e'])
(4, ['c'])
(3, ['c', 'd'])
(2, ['c', 'd', 'e'])
(3, ['c', 'e'])
(3, ['d'])
(2, ['d', 'e'])
(3, ['e'])


## Prefix 알고리즘 프로세스
1. 가능한 모든 1개 상품 시퀀스를 나열한 후 각각의 발생 빈도 계산
  - 특정 항목의 발생 빈도는 지지도 카운트(Support Count)
  - 연관 규칙 마이닝처럼 최소 지지도 이하의 상품을 탈락
    - PrefixSpan 알고리즘에서는 지지도 카운트를 기준으로 삼음
    - 최소 지지도 카운트 2로 설정
2. 1단계를 통과한 각 상품을 Prefix로 두고 Prefix 별로 탐색 영역을 분리
3. 각 Prefix 별로 Projected DB를 정리하고 그 안에서 탐색하다보면 
4. 각 Prefix별로 계속해서 Projected DB를 점점 축소해나가

In [40]:
# 데이터 불러오기
df = pd.read_csv('retail_data.csv')
df.head()

,OrderID,StockCode,ProdName,Quantity,OrderDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0


In [41]:
df = df.sort_values('ProdName')

In [42]:
df2 = df.groupby(['CustomerID', 'OrderDate', 'OrderID'])['ProdName'].apply(tuple).reset_index()

In [43]:
df2 = df2.sort_values('OrderDate')

In [44]:
sequence_df = df2.groupby(['CustomerID'])['ProdName'].apply(list).reset_index()
sequence_df.head()

,CustomerID,ProdName
0,12427.0,"[(6 RIBBONS RUSTIC CHARM, BALLOONS WRITING SE..."
1,12431.0,"[(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKE..."
2,12433.0,"[(20 DOLLY PEGS RETROSPOT, 200 RED + WHITE BEN..."
3,12583.0,"[( SET 2 TEA TOWELS I LOVE LONDON , ALARM CLOC..."
4,12662.0,"[(3 HOOK HANGER MAGIC GARDEN, 5 HOOK HANGER MA..."


In [45]:
# 라이브러리 불러오기
from prefixspan import PrefixSpan

In [46]:
ps = PrefixSpan(sequence_df['ProdName'])

In [47]:
ps.frequent(2)  # 지지도 카운트가 2 이상인 패턴 

[(2, [('BATH BUILDING BLOCK WORD',)]),
 (2, [("PAPER CHAIN KIT 50'S CHRISTMAS ",)]),
 (2, [('JAM MAKING SET PRINTED', 'JAM MAKING SET WITH JARS')])]

In [48]:
ps.topk(3)  # 가장 지지도가 높은 상위 3개의 패턴 확인

[(2, [('BATH BUILDING BLOCK WORD',)]),
 (2, [('JAM MAKING SET PRINTED', 'JAM MAKING SET WITH JARS')]),
 (2, [("PAPER CHAIN KIT 50'S CHRISTMAS ",)])]

## PrefixSpan 알고리즘 직접 구현

In [49]:
class PrefixSpan:
    def __init__(self, min_support):
        self.min_support = min_support

    def run(self, sequences):
        frequent_patterns = []
        self.prefix_span([], sequences, frequent_patterns)
        return self.to_dataframe(frequent_patterns)

    def prefix_span(self, prefix, projected_db, frequent_patterns):
        frequent_items = self.get_frequent_items(projected_db)
        for item, support in frequent_items.items():
            new_prefix = prefix + [item]
            frequent_patterns.append((new_prefix, support))
            new_projected_db = self.build_projected_db(projected_db, item)
            if new_projected_db:
                self.prefix_span(new_prefix, new_projected_db, frequent_patterns)

    def get_frequent_items(self, projected_db):
        item_counts = {}
        for sequence in projected_db:
            visited = set()
            for itemset in sequence:
                for item in itemset:
                    if item not in visited:
                        if item not in item_counts:
                            item_counts[item] = 0
                        item_counts[item] += 1
                        visited.add(item)
        return {item: count for item, count in item_counts.items() if count >= self.min_support}

    def build_projected_db(self, projected_db, item):
        new_projected_db = []
        for sequence in projected_db:
            for idx, itemset in enumerate(sequence):
                if item in itemset:
                    new_projected_db.append(sequence[idx+1:])
                    break
        return new_projected_db

    def to_dataframe(self, frequent_patterns):
        pattern_data = [{'item': pattern[0], 'support_count': pattern[1], 'item_count': len(pattern[0])} for pattern in frequent_patterns]
        df_patterns = pd.DataFrame(pattern_data)
        return df_patterns

In [50]:
ps = PrefixSpan(min_support=2)  # 최소 지지도 카운트 설정

In [51]:
patterns = ps.run(sequence_df['ProdName'])
patterns.head()

,item,support_count,item_count
0,[6 RIBBONS RUSTIC CHARM],9,1
1,[BALLOONS WRITING SET ],6,1
2,[CHILDS BREAKFAST SET CIRCUS PARADE],4,1
3,[CHILDS BREAKFAST SET SPACEBOY ],9,1
4,[COFFEE MUG CAT + BIRD DESIGN],2,1


- support_count : 해당 아이템이 전체 트랜잭션에서 몇 번 등장했는지
- item_count : 몇 건의 트랜잭션에서 등장했는지

**트랜잭션과 아이템**

- 트랜잭션 (Transaction) : 구매 기록이나 장바구니와 같은 단위. 한 번의 구매나 이벤트

- 아이템 (Item) : 트랜잭션 내에 포함된 상품이나 항목.


e.g. 트랜잭션 - 셔츠, 신발, 모자

In [52]:
patterns['item_count'].unique()

array([1, 2, 3])

In [53]:
patterns.query('item_count >= 2').sort_values(by='support_count', ascending=False).head()

,item,support_count,item_count
68,"[PACK OF 72 RETROSPOT CAKE CASES, CLOTHES PEGS...",2,2
112,"[ROTATING SILVER ANGELS T-LIGHT HLDR, CLOTHES ...",2,2
167,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE HAN...",2,2
168,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE HAN...",2,3
169,"[WHITE HANGING HEART T-LIGHT HOLDER, RED WOOLL...",2,2


**장바구니 분석 프로세스**
1. 데이터 전처리

2. 규칙 및 패턴 추출

  1) 연관 규칙 마이닝 : 상호 연관성 - 빈발 항목 집합 (Frequent Itemset)

- 알고리즘 예) Apriori, FP-Growth

  2) 순차 패턴 마이닝 : 시간, 순서 - 빈발 시퀀스 (Frequent Sequence)

-  알고리즘 예) PrefixSpan

3. 평가 지표 활용

  1) 연관 규칙 마이닝 : 지지도, 신뢰도, 향상도

  2) 순차 패턴 마이닝 : 지지도 카운트